In [56]:
import pandas as pd
pd.reset_option('display.float_format')
pd.options.display.float_format = '{:.2f}'.format

df_ffca= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\FFCA.xlsx")
df_qv = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\QV.xlsx")
df_stock = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\stock.xlsx") 
df_stock.drop([0])
df_inventory = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\inventory.xlsx")
df_CS = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\CS_01.xlsx")
df_CP = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\CP_approved.xlsx")


In [57]:
#delete nan sales
df_ffca.dropna(subset=['Sales Assignment ID '], inplace=True)
# 一、先将字段拆分
df_CS['Assignments']=df_CS['Assignments'].map(lambda x:x.split(','))
#二、然后直接调用explode()方法
df_CS_explode = df_CS.explode('Assignments')


In [72]:
#column filter
cols_ffca = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
        'Sales Invoiced Amount', 'Sales Settled Amount']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator']
cols_inventory = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'EXPECTED RECEIPT DATE']
cols_stock = ['Inventory Id','Origin Country']
cols_cs = ['ID', 'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank', 'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount']
cols_ffca_blend = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
        'Sales Invoiced Amount', 'Sales Settled Amount', 'QTY']

df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_CS_f1 = pd.DataFrame(df_CS_explode, columns=cols_cs)
df_ffca_f1['Current Value'] = pd.to_numeric(df_ffca_f1['Current Value'], errors='coerce')
df_ffca_f1['Authorised Paid Quantity'] = pd.to_numeric(df_ffca_f1['Authorised Paid Quantity'], errors='coerce')
df_ffca_f1['Financed Quantity'] = pd.to_numeric(df_ffca_f1['Financed Quantity'], errors='coerce')

def blend(a):
    return a == "Blend"

#blend
bdf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca_blend)
df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_inventory_f1 = df_inventory_f1.drop(row_list)

bffca_qv = pd.merge(bdf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
bffca_qv_cp = pd.merge(bffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
binventory_stock = pd.merge(df_blend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
FQC = pd.merge(bffca_qv_cp, df_CS_f1, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
bfinal_sales = pd.merge(FQC, binventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])

def function(a, b, c):
    c * (1 + (a-b)/a)

bfinal_sales['Purchased Wet Weight']  = bfinal_sales.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_sales = bfinal_sales.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])


ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
ffca_qv_cp = pd.merge(ffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
inventory_stock = pd.merge(df_inventory_f1, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
FQC = pd.merge(ffca_qv_cp, df_CS_f1, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
nfinal_sales = pd.merge(FQC, inventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])



In [75]:
bfinal_sales.columns

Index(['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
       'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
       'Sales Invoice ID', 'Estimated Sales Valuation',
       'Purchase Settled Amount', 'Authorised Paid Quantity',
       'Financed Quantity', 'Current Value', 'Treasury Payment Date',
       'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
       'Sales Settled Amount', 'Assignment ID', 'Group Company', 'Trade ID',
       'Quota ID', 'Traffic Operator', 'Sales Assignment Counterparty',
       'Sales Contractual Counterparty Open Account Approved ', 'ID',
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
       'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
       'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'QTY', 'Purchased Wet Weight',
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country'],
      dtype='object')

In [76]:
bfinal_sales.columns

order = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity','Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
       'Purchase Settled Amount', 'Authorised Paid Quantity','Financed Quantity', 'Current Value', 'Treasury Payment Date','Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount','Sales Settled Amount',  \
        'Assignment ID', 'Group Company','Trade ID', 'Quota ID', 'Traffic Operator','Sales Assignment Counterparty','Sales Contractual Counterparty Open Account Approved ', 'ID', \
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank','Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'QTY', 'Purchased Wet Weight',\
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

bfinal_sales = bfinal_sales[order]

bfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
       'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
       'Sales Invoice ID', 'Estimated Sales Valuation',
       'Purchase Settled Amount', 'Authorised Paid Quantity',
       'Financed Quantity', 'Current Value', 'Treasury Payment Date',
       'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
       'Sales Settled Amount', 'Assignment ID', 'Group Company', 'Trade ID',
       'Quota ID', 'Traffic Operator', 'Sales Assignment Counterparty',
       'Sales Contractual Counterparty Open Account Approved ', 'ID',
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
       'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
       'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

final_sales = pd.concat([nfinal_sales,bfinal_sales])

In [59]:
#Comparison between invoice amount and current value

def function1(a,b,c):
    if 3 * (-b) >= a:
        return (-b) - c
    else:
        return 0

def function2(a,b,c):
    if 3 * (-b) < a:
        return (-b) - c
    else:
        return 0

final_sales['Unpaid Provisional Receivables(USD)'] = final_sales.apply(lambda x: function1(x['Estimated Sales Valuation'], x['Sales Invoiced Amount'], x['Sales Settled Amount']), axis = 1)
final_sales['Unpaid Final Receivables(USD)'] = final_sales.apply(lambda x: function2(x['Estimated Sales Valuation'], x['Sales Invoiced Amount'], x['Sales Settled Amount']), axis = 1)
final_sales.loc[:,"Uninvoiced Final Receivables(USD)"] = final_sales["Current Value"] - final_sales["Sales Invoiced Amount"]
final_sales.loc[:,"SCR Covered By LC (SCR-Sales Contractual Rights)"] = final_sales["Current Value"] - final_sales["Sales Invoiced Amount"]

In [60]:
#Comparison between invoice amount and LC current balance
def function3(a,b):
    if a > b:
        return a
    else:
        return 0

def function4(a,b):
    if a < b:
        return b - a
    else:
        return 0

def function5(a,b):
    if 3 * (-b) >= a:
        return "Prov"
    else:
        return "Final"

def function6(a):
    if a == "Final":
        return "100%"
    else:
        return "90%"


final_sales['Total Financeable Sales Invoices Covered by LC']  = final_sales.apply(lambda x: function3(x['Sales Invoiced Amount'], x['Balance Amount']), axis = 1)
final_sales['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function4(x['Sales Invoiced Amount'], x['Balance Amount']), axis = 1)
final_sales['Sales Provisional Final']  = final_sales.apply(lambda x: function5(x['Sales Invoiced Amount'], x['Balance Amount']), axis = 1)
final_sales['Sales Provisional Percentage']  = final_sales.apply(lambda x: function6(x['Sales Provisional Final']), axis = 1)
final_sales.loc[:, 'SCR Covered By LC (SCR-Sales Contractual Rights)'] = final_sales['Uninvoiced Final Receivables(USD)']

In [61]:
import numpy as np

#adding 
final_sales['Sales Provisional Number'] = '1'
final_sales[['Authorised Paid Quantity', 'Financed Quantity']] = final_sales[['Authorised Paid Quantity', 'Financed Quantity']].astype('float')
final_sales.loc[:,"Financing Ratio"] = final_sales["Authorised Paid Quantity"]/final_sales["Financed Quantity"]
final_sales['Sales Invoice Date'] = '0'
final_sales['Sales Invoice Due Date'] = '0'
final_sales['Sales Invoice Days Overdue'] = '0'
final_sales['Cost Code ID'] = 'I01'
final_sales['Invoice Line Quantity'] = '0'
final_sales['Invoice Line Quantity UOM'] = 'MT'
final_sales['Purchase Dry Weight UOM'] = 'MT'
final_sales['Purchase Wet Weight UOM'] = 'MT'
final_sales['Receipt In Progress'] = 'N'
final_sales['Delivery In Progress'] = 'N'
final_sales['Summary Outstanding Receivables(USD)'] = final_sales['Uninvoiced Final Receivables(USD)']
final_sales['Sales Invoice Counterparty'] = final_sales['Counterparty of Sales']
final_sales['Eligible Dry Weight'] = final_sales['DRY WEIGHT']
final_sales['Eligible Wet Weight'] = final_sales['WET WEIGHT']

def function7(a, b):
    if b == 0:
        return 0
    else:
        return -a/b
        
def function8(a):
    if a>1:
        return 1
    else:
        return a
        


final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function7(x['Estimated Sales Valuation'], x['Sales Invoiced Amount']), axis = 1)

final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function8(x['Sales Assignment Eligible Percentage']), axis = 1)

final_sales.columns


Index(['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
       'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
       'Sales Invoice ID', 'Estimated Sales Valuation',
       'Purchase Settled Amount', 'Authorised Paid Quantity',
       'Financed Quantity', 'Current Value', 'Treasury Payment Date',
       'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
       'Sales Settled Amount', 'Assignment ID', 'Group Company', 'Trade ID',
       'Quota ID', 'Traffic Operator', 'Sales Assignment Counterparty',
       'Sales Contractual Counterparty Open Account Approved ', 'ID',
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
       'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
       'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT',
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country',
       'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(U

In [62]:
final_sales = final_sales.drop(columns=['Inventory ID', 'Current Value','Purchase Settled Amount', 'Purchase Invoice', 'Purchase Invoiced Amount', 'Current Value', 'Treasury Payment Date', \
    'Assignment ID', 'Sales Assignment Counterparty', 'Assignments', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', \
        'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country'], axis=1)


In [63]:
final_sales.columns = ['Purchase Assignment Reference', 'Sales Assignment Reference', 'Purchase Commodity', 'Purchase Contractual Counterparty', \
    'Sales Assignment Counterparty', 'Sales Invoice ID', 'Cost Amount(USD)', 'Authorized Paid Quantity', 'Financed Quantity', 'Invoice Line Amount(USD)', 'Paid Amount', 'Purchase Group Company Code', \
        'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Sales Contractual Counterparty Open Account Approved', 'LC Reference', \
            'Sales Credit Security Type','LC Opening Bank', 'LC Confirming Bank', 'LC Issuance Date', 'LC Expiry Date', 'LC Current Balance', 'LC Original Amount', \
                'Purchase Dry Weight', 'Purchase Wet Weight', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)',
                    'Uninvoiced Final Receivables(USD)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Total Financeable Sales Invoices Covered by LC', \
                        'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'Sales Provisional Final', 'Sales Provisional Percentage', \
                            'Sales Provisional Number', 'Financing Ratio', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', \
                                'Cost Code ID', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Purchase Dry Weight UOM', 'Purchase Wet Weight UOM', 'Receipt In Progress', \
                                    'Delivery In Progress', 'Summary Outstanding Receivables(USD)', 'Sales Invoice Counterparty','Eligible Dry Weight', 'Eligible Wet Weight', 'Sales Assignment Eligible Percentage']

In [64]:
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
    'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
        'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
            'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
                'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
                    'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                        'Authorized Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                            'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                                'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

In [65]:
final_sales_test = final_sales[order]
input = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\input.xlsx")
result = pd.merge(final_sales_test, input, left_on=["Purchase Assignment Reference"], right_on=["assignment"], how='right')
result = result.drop_duplicates(['Purchase Assignment Reference'])
# result.to_excel(r"C:\Users\Yuyan.Lu\Desktop\\test5.xlsx")

In [66]:
import numpy as np
test_p = result.copy(deep=True)

test_p['Purchase Contractual Counterparty'] = np.nan
test_p['Sales Credit Security Type'] = np.nan
test_p['LC Reference'] = np.nan
test_p['LC Original Amount'] = np.nan
test_p['LC Current Balance'] = np.nan
test_p['LC Issuance Date'] = np.nan
test_p['LC Expiry Date'] = np.nan
test_p['LC Opening Bank'] = np.nan
test_p['LC Confirming Bank'] = np.nan
test_p['Total Financeable Sales Invoices Covered by LC'] = np.nan
test_p['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = np.nan
test_p['SCR Covered By LC (SCR-Sales Contractual Rights)'] = np.nan
test_p['Cost Amount(USD)'] = np.nan
test_p['Authorized Paid Quantity'] = np.nan
test_p['Financed Quantity'] = np.nan
test_p['Financing Ratio'] = np.nan
test_p['Uninvoiced Final Receivables(USD)'] = np.nan
test_p['Summary Outstanding Receivables(USD)'] = np.nan
test_p['Sales Contractual Counterparty Open Account Approved '] = np.nan
test_p['Purchase Dry Weight'] = np.nan
test_p['Purchase Dry Weight UOM'] = np.nan
test_p['Purchase Wet Weight'] = np.nan
test_p['Purchase Wet Weight UOM'] = np.nan
test_p['Receipt In Progress'] = np.nan
test_p['Delivery In Progress'] = np.nan
test_p['Eligible Dry Weight'] = np.nan
test_p['Eligible Wet Weight'] = np.nan

test_q = result.copy(deep=True)
test_q['Sales Invoice ID'] = np.nan
test_q['Sales Provisional Final '] = np.nan
test_q['Sales Provisional Number'] = np.nan
test_q['Sales Provisional Percentage'] = np.nan
test_q['Sales Invoice Date'] = np.nan
test_q['Sales Invoice Due Date'] = np.nan
test_q['Sales Invoice Days Overdue'] = np.nan
test_q['Sales Invoice Counterparty'] = np.nan
test_q['Cost Code ID'] = np.nan
test_q['Invoice Line Quantity'] = np.nan
test_q['Invoice Line Quantity UOM'] = np.nan

test_f = pd.concat([test_p,test_q])
test_f = test_f.sort_values(by=['Purchase Assignment Reference'], ascending=False)